In [1]:
import pandas as pd

In [2]:
submits = pd.read_csv('joined.csv')

In [3]:
uniq_submits = submits.drop_duplicates(subset=('authorid', 'problemid'), keep='last')

In [4]:
counts = uniq_submits.groupby('authorid').count()

In [5]:
interesting_users = counts[counts.problemid > 20]

In [207]:
int_users = set(interesting_users.index)

In [208]:
len(int_users)

16956

In [8]:
int_submits_ind = uniq_submits.authorid.apply(lambda x: x in int_users)

In [9]:
interesting_submits = uniq_submits[int_submits_ind.values]

In [189]:
interesting_submits.count()

submitid     1417691
authorid     1417691
problemid    1417691
dtype: int64

In [196]:
import numpy as np

In [230]:
test_users = np.random.choice(list(int_users), size=(int(0.4 * len(int_users))), replace=False)

In [231]:
len(test_users)

6782

In [232]:
test_subs_s = set()

In [233]:
for user in test_users:
  subs = interesting_submits[interesting_submits.authorid == user].head(10)
  for sub in subs.submitid:
    test_subs_s.add(sub)

In [234]:
len(test_subs_s)

67820

In [235]:
test_subs = interesting_submits[interesting_submits.submitid.apply(lambda x: x in test_subs_s).values]

In [237]:
train_subs = interesting_submits[interesting_submits.submitid.apply(lambda x: x not in test_subs_s).values]

In [30]:
import turicreate as tc

In [31]:
tc_data = tc.SFrame(interesting_submits)

In [239]:
# tc_train, tc_test = tc.recommender.util.random_split_by_user(tc_data, user_id='authorid', item_id='problemid')
tc_train = tc.SFrame(train_subs)
tc_test = tc.SFrame(test_subs)

In [240]:
model = tc.item_similarity_recommender.create(tc_train, user_id='authorid', item_id='problemid', item_data=tc.SFrame(complexities))

Warning: Column 'submitid' ignored.

To use one of these as a rating column, specify the column name to be used as target submitid" and use a method that allows the use of a target.

Preparing data set.

Data has 1349871 observations with 16956 users and 1148 items.

Data prepared in: 0.517759s

...

| 269.74ms                            | 100              | 1148            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.336082s

In [241]:
model.evaluate(tc_test)

recommendations finished on 1000/6782 queries. users per second: 101368

recommendations finished on 2000/6782 queries. users per second: 120286

recommendations finished on 3000/6782 queries. users per second: 132100

recommendations finished on 4000/6782 queries. users per second: 138461

recommendations finished on 5000/6782 queries. users per second: 142264

recommendations finished on 6000/6782 queries. users per second: 145691


Precision and recall summary statistics by cutoff


+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.280153347095252  | 0.028015334709525087 |
|   2    | 0.27219109407254505 | 0.05443821881450896  |
|   3    |  0.2589698220780493 | 0.07769094662341483  |
|   4    |  0.2504423473901503 | 0.10017693895606015  |
|   5    |  0.2432025951046888 |  0.1216012975523444  |
|   6    | 0.23616435663029592 |  0.1416986139781778  |
|   7    | 0.22865147238488448 | 0.16005603066941906  |
|   8    |  0.220749778826305  |  0.1765998230610441  |
|   9    |  0.2149972148497656 |  0.1934974933647891  |
|   10   | 0.20980536714833398 | 0.20980536714833398  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]



{'precision_recall_by_user': Columns:
 	authorid	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 122076
 
 Data:
 +----------+--------+---------------------+--------+-------+
 | authorid | cutoff |      precision      | recall | count |
 +----------+--------+---------------------+--------+-------+
 |  287903  |   1    |         0.0         |  0.0   |   10  |
 |  287903  |   2    |         0.5         |  0.1   |   10  |
 |  287903  |   3    |  0.3333333333333333 |  0.1   |   10  |
 |  287903  |   4    |         0.25        |  0.1   |   10  |
 |  287903  |   5    |         0.2         |  0.1   |   10  |
 |  287903  |   6    | 0.16666666666666666 |  0.1   |   10  |
 |  287903  |   7    | 0.14285714285714285 |  0.1   |   10  |
 |  287903  |   8    |         0.25        |  0.2   |   10  |
 |  287903  |   9    |  0.2222222222222222 |  0.2   |   10  |
 |  287903  |   10   |         0.2         |  0.2   |   10  |
 +----------+--------+---------------------+--------+-----

In [277]:
tc.ranking_factorization_recommender.create?

Signature:
tc.ranking_factorization_recommender.create(
    observation_data,
    user_id='user_id',
    item_id='item_id',
    target=None,
    user_data=None,
    item_data=None,
    num_factors=32,
    regularization=1e-09,
    linear_regularization=1e-09,
    side_data_factorization=True,
    ranking_regularization=0.25,
    unobserved_rating_value=None,
    num_sampled_negative_examples=4,
    max_iterations=25,
    sgd_step_size=0,
    random_seed=0,
    binary_target=False,
    solver='auto',
    verbose=True,
    **kwargs,
)
Docstring:
Create a RankingFactorizationRecommender that learns latent factors for each
user and item and uses them to make rating predictions.

Parameters
----------
observation_data : SFrame
    The dataset to use for training the model. It must contain a column of
    user ids and a column of item ids. Each row represents an observed
    interaction between the user and the item.  The (user, item) pairs
    are stored with the model so that they can late

In [311]:
model2 = tc.ranking_factorization_recommender.create(
  tc_train,
  user_id='authorid',
  item_id='problemid',
  solver='ials',
  linear_regularization=1,
  binary_target=True)

Preparing data set.

Data has 1349871 observations with 16956 users and 1148 items.

Data prepared in: 0.544699s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

| 23      | 3.00s        | 0.35698                   |

| 24      | 3.13s        | 0.356976                  |

| FINAL   | 3.13s        | 0.356976                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

In [312]:
model2.evaluate(tc_test)

recommendations finished on 1000/6782 queries. users per second: 59530.9

recommendations finished on 2000/6782 queries. users per second: 90975.3

recommendations finished on 3000/6782 queries. users per second: 112880

recommendations finished on 4000/6782 queries. users per second: 127902

recommendations finished on 5000/6782 queries. users per second: 139938

recommendations finished on 6000/6782 queries. users per second: 148302


Precision and recall summary statistics by cutoff


+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.28959009141846015 | 0.028959009141845966 |
|   2    | 0.28472427012680646 | 0.05694485402536118  |
|   3    |  0.2817261378157865 | 0.08451784134473608  |
|   4    |  0.2770200530816859 | 0.11080802123267477  |
|   5    | 0.27210262459451484 | 0.13605131229725742  |
|   6    |  0.267079524230807  | 0.16024771453848424  |
|   7    | 0.26155369254750005 | 0.18308758478325002  |
|   8    |  0.2556399292244175 | 0.20451194337953402  |
|   9    | 0.25056522166519196 | 0.22550869949867264  |
|   10   | 0.24500147449130072 | 0.24500147449130072  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]



{'precision_recall_by_user': Columns:
 	authorid	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 122076
 
 Data:
 +----------+--------+--------------------+--------+-------+
 | authorid | cutoff |     precision      | recall | count |
 +----------+--------+--------------------+--------+-------+
 |  287903  |   1    |        1.0         |  0.1   |   10  |
 |  287903  |   2    |        0.5         |  0.1   |   10  |
 |  287903  |   3    | 0.6666666666666666 |  0.2   |   10  |
 |  287903  |   4    |        0.5         |  0.2   |   10  |
 |  287903  |   5    |        0.4         |  0.2   |   10  |
 |  287903  |   6    | 0.3333333333333333 |  0.2   |   10  |
 |  287903  |   7    | 0.2857142857142857 |  0.2   |   10  |
 |  287903  |   8    |        0.25        |  0.2   |   10  |
 |  287903  |   9    | 0.3333333333333333 |  0.3   |   10  |
 |  287903  |   10   |        0.3         |  0.3   |   10  |
 +----------+--------+--------------------+--------+-------+
 [122076 r

In [13]:
complexities = pd.read_csv('complexities.csv')

In [26]:
complexities = complexities.sort_values(by='compexity', axis=0)

In [29]:
complexities = complexities.rename(columns={'compexity': 'complexity'})

In [49]:
compls = complexities.set_index('problemid')

In [70]:
tc_train['authorid']

dtype: int
Rows: 1401260
[308259, 236293, 243780, 271442, 271442, 308259, 271442, 271442, 297970, 291729, 222188, 271442, 271442, 272108, 271442, 287903, 271442, 308259, 287903, 310962, 296658, 287903, 310962, 287903, 113890, 295965, 287903, 287903, 282708, 283212, 310962, 310962, 310962, 310962, 296321, 308259, 163199, 272749, 310962, 272749, 288653, 272749, 310962, 310962, 310962, 310962, 310962, 310962, 310962, 310962, 310962, 287818, 287818, 287818, 287818, 287818, 287818, 287818, 220430, 301319, 220430, 275158, 292197, 151151, 301319, 220430, 309330, 310052, 222188, 275158, 304413, 225600, 296658, 287818, 304413, 222188, 310962, 310962, 15158, 264239, 283212, 310962, 310962, 310962, 121937, 310962, 310962, 15158, 310962, 140391, 279371, 310962, 279371, 279371, 140391, 310962, 140391, 140391, 140391, 270019, ... ]

In [73]:
arr = tc_train[tc_train['authorid'] == 248409]['problemid']

In [78]:
class ComplexityModel:
  def __init__(self, complexities, train):
    self._compls = complexities
    self._train = train
    
  def recommend_user(self, userid, k=10):
    return self.recommend_problems(self._train[self._train['authorid'] == userid]['problemid'].to_numpy(), k=k)
  
  def recommend_problems(self, problems, k=10):
    solved = set(problems)
    result = []
    for row_item in self._compls.iterrows():
      id_, row = row_item
      if row.problemid not in solved:
        result.append(row.problemid)
        if len(result) >= k:
          break
    return result

In [79]:
easy_model = ComplexityModel(complexities, tc_train)

In [83]:
model.save('model')

In [88]:
model.recommend_from_interactions(tc_train[tc_train['authorid'] == 248409]['problemid'])

problemid,score,rank
1082,0.14300597172517043,1
1084,0.14250278434692285,2
1740,0.10978048428511009,3
1192,0.09930944175292285,4
1353,0.09249867575290875,5
1864,0.0889982554392937,6
1012,0.08504603726741596,7
1572,0.08463931924257523,8
2000,0.08374470243087181,9
1017,0.08230161781494434,10


In [98]:
model.recommend(users=[248409], k=1000).tail()

authorid,problemid,score,rank
248409,1949,0.0,983
248409,1169,0.0,984
248409,1622,0.0,985
248409,1396,0.0,986
248409,1398,0.0,987
248409,1550,0.0,988
248409,1375,0.0,989
248409,1621,0.0,990
248409,1759,0.0,991
248409,1314,0.0,992


In [100]:
NASTYA = 248409

In [244]:
subs = interesting_submits[interesting_submits['authorid'] == NASTYA]

In [258]:
n_test_subs = subs.problemid[:10][::-1]


In [246]:
n_val_subs = subs.problemid[10:]

In [247]:
model_recs = model.recommend_from_interactions(n_val_subs.to_list(), k=10)

In [182]:
model2_recs = model2.recommend_from_interactions(tc.SFrame(subs[10:])['submitid', 'problemid'], k=10)

In [248]:
easy_recs = easy_model.recommend_problems(n_val_subs, k=10)

In [176]:
def compute_apk(y_true, y_pred, k=10):
    """
    average precision at k, y_pred is assumed 
    to be truncated to length k prior to feeding
    it to the function
    """
    # convert to set since membership 
    # testing in a set is vastly faster
    actual = set(y_true)
    
    # precision at i is a percentage of correct 
    # items among first i recommendations; the
    # correct count will be summed up by n_hit
    n_hit = 0
    precision = 0
    for i, p in enumerate(y_pred, 1):
        if p in actual:
            n_hit += 1
            precision += n_hit / i

    # divide by recall at the very end
    avg_precision = precision / min(len(actual), k)
    return avg_precision

In [260]:
compute_apk(n_test_subs, model_recs['problemid'].to_numpy())

0.5666666666666667

In [183]:
compute_apk(test_subs, model2_recs['problemid'].to_numpy())

0.12222222222222223

In [261]:
compute_apk(n_test_subs, easy_recs)

0.6366666666666667

In [262]:
from tqdm import trange

In [265]:
mapk = 0
for i in trange(len(test_users)):
  user = test_users[i]
  user_subs = test_subs[test_subs.authorid == user].problemid
  user_train_subs = train_subs[train_subs.authorid == user].problemid
  model_recs = model.recommend_from_interactions(user_train_subs.to_list(), k=10)
  mapk += compute_apk(user_subs, model_recs['problemid'].to_numpy())
mapk /= len(test_users)

100%|██████████| 6782/6782 [03:27<00:00, 32.65it/s]


In [313]:
mapk = 0
for i in trange(len(test_users)):
  user = test_users[i]
  user_subs = test_subs[test_subs.authorid == user].problemid
  user_train_subs = train_subs[train_subs.authorid == user]
  model_recs = model2.recommend_from_interactions(tc.SFrame(user_train_subs), k=10)
  mapk += compute_apk(user_subs, model_recs['problemid'].to_numpy())
mapk /= len(test_users)

100%|██████████| 6782/6782 [03:35<00:00, 31.50it/s]


In [314]:
mapk

0.14885138883037666

In [315]:
mapk = 0
for i in trange(len(test_users)):
  user = test_users[i]
  user_subs = test_subs[test_subs.authorid == user].problemid
  user_train_subs = train_subs[train_subs.authorid == user].problemid
  model_recs = easy_model.recommend_problems(user_train_subs.to_list(), k=10)
  mapk += compute_apk(user_subs, model_recs)
mapk /= len(test_users)

100%|██████████| 6782/6782 [00:38<00:00, 174.90it/s]


In [316]:
mapk

0.10099861093557606

In [317]:
model2.save('prod_model')

In [319]:
!du -hs prod_model/

11M	prod_model/


In [321]:
tc.SFrame?

Init signature: tc.SFrame(data=None, format='auto', _proxy=None)
Docstring:     
SFrame means scalable data frame. A tabular, column-mutable dataframe object that can
scale to big data. The data in SFrame is stored column-wise, and is
stored on persistent storage (e.g. disk) to avoid being constrained by
memory size.  Each column in an SFrame is a size-immutable
:class:`~turicreate.SArray`, but SFrames are mutable in that columns can be
added and subtracted with ease.  An SFrame essentially acts as an ordered
dict of SArrays.

Currently, we support constructing an SFrame from the following data
formats:

* csv file (comma separated value)
* sframe directory archive (A directory where an sframe was saved
  previously)
* general text file (with csv parsing options, See :py:meth:`read_csv()`)
* a Python dictionary
* pandas.DataFrame
* JSON

and from the following sources:

* your local file system
* a network file system mounted locally
* HDFS
* Amazon S3
* HTTP(S).

Only basic examples o

In [320]:
tc.load_model

Signature:
tc.recommender.factorization_recommender.create(
    observation_data,
    user_id='user_id',
    item_id='item_id',
    target=None,
    user_data=None,
    item_data=None,
    num_factors=8,
    regularization=1e-08,
    linear_regularization=1e-10,
    side_data_factorization=True,
    nmf=False,
    binary_target=False,
    max_iterations=50,
    sgd_step_size=0,
    random_seed=0,
    solver='auto',
    verbose=True,
    **kwargs,
)
Docstring:
Create a FactorizationRecommender that learns latent factors for each
user and item and uses them to make rating predictions. This includes
both standard matrix factorization as well as factorization machines models
(in the situation where side data is available for users and/or items).

Parameters
----------
observation_data : SFrame
    The dataset to use for training the model. It must contain a column of
    user ids and a column of item ids. Each row represents an observed
    interaction between the user and the item.  The (